# 2. 自然言語処理と単語の分散表現

この本では、自然言語処理として、（言葉の意味の最小単位である）単語の意味をコンピュータに理解させる方法として、以下の３つの方法を見ていく。

* シソーラスによる手法
* カウントベースによる手法
* 推論ベースの手法（word2vec) → ３章

## 2.2 シソーラス

シソーラスとは、基本的には類語辞書で、同義語や類義語が同じグループに分類される辞書で、以下の特徴を持つ。

* 人の手でメンテナンスされる辞書。
* 例えば、`car ` = `auto` `automobile` `machine` `motorcar` のよう同義語を得ることができる。
* 単語間で、「上位と下位」、「全体と部分」などの、関連性が定義されていることがある。例えば、carの場合

![relation](https://raw.githubusercontent.com/aha-oretama/deep-learning-from-scratch-2/master/chapter2/image/thesaurus.png)

このようにすべての単語に対して、類義語の集合を作り、それぞれの単語の関係をグラフで表現することで、単語間のつながりを定義できる。  
これは、コンピュータに単語の意味を（間接的にであれ）授けることができたと言える。

### 2.2.1 WordNet

自然言語処理の分野で、最も有名なシソーラスはWordNetであり、NLTKに入っている。  
http://www.nltk.org/howto/wordnet.html

ただの辞書なので動作は割愛。実際の動作は付録B参照。

### 2.2.2 シソーラスの問題点

人の手でメンテナンスされるもののため、以下の問題点が存在する。

* 時代の変化に対応するのが困難  
→ 言葉は時とともに変化する。
* 人の作業コストが高い
* 単語のニュアンスを表現できない  
→ ex. ヴィンテージとレトロ

## 2.3 カウントベースの手法


コーパス → 分布仮説からの共起行列 → コサイン類似度 → 相互情報量 → 次元削減という流れ。  
また１文からなる単純なテキスト → 実践的なコーパスという流れ。

### 2.3.1 コーパスの下準備

コーパスとは、自然言語処理の研究やアプリケーションのために目的をもって収集された大量のテキストデータ。  
以降は、まず`You say goodbye and I say hello.`という文を対象として、処理を行っていく。ここでは、下準備処理を行う。


In [11]:
import numpy as np

def preprocess(text):
    text = text.lower()
    text = text.replace('.',' .')
    words = text.split(' ')
    
    word_to_id = {}
    id_to_word = {}
    for word in words:
        if word not in word_to_id:
            new_id = len(word_to_id)
            word_to_id[word] = new_id
            id_to_word[new_id] = word
    
    corpus = np.array([word_to_id[w] for w in words])
    
    return corpus, word_to_id, id_to_word

text = 'You say goodbye and I say hello.'
corpus, word_to_id, id_to_word = preprocess(text)

print(corpus) ## 単語IDのリスト  
print(word_to_id) ## 単語から単語IDへのディクショナリ 
print(id_to_word) ## 単語IDから単語へのディクショナリ

[0 1 2 3 4 1 5 6]
{'you': 0, 'say': 1, 'goodbye': 2, 'and': 3, 'i': 4, 'hello': 5, '.': 6}
{0: 'you', 1: 'say', 2: 'goodbye', 3: 'and', 4: 'i', 5: 'hello', 6: '.'}


### 2.3.2 単語の分散表現

自然言語処理で、単語を（色のRGBのような）ベクトル表現で行うことができれば、定量化が簡単に行える。  
単語のベクトルを、単語の分散表現と呼ぶ。

### 2.3.3 分布仮説

自然言語処理の歴史において、単語をベクトルで表す研究は数多く行われてきた。そのほどんどすべてが、「単語の意味は、周囲の単語によって形成される」という、分布仮説に呼ばれるもの。  
つまり、単語の意味はその単語のコンテキスト（文脈）によって、単語の意味が形成されるということ。

ex.   
「I drink beer」「We drink wine」  
「I guzzle beer」「We　guzzle wine」  
guzzle: がぶがぶ飲む

この本では、以下のように定義している。

コンテキスト： ある中央の単語に対して、その周囲にある単語
ウィンドウサイズ： 周囲の単語をどれだけ含めるか

![context](https://raw.githubusercontent.com/aha-oretama/deep-learning-from-scratch-2/master/chapter2/image/context.png)

### 2.3.4 共起行列

カウントベースでは、周囲の単語をカウントすることで、ベクトルを表現する。  
具体的にはある単語に着目した場合、その周囲にどのような単語がどれだけ現れるのかをカウントし、それを集計する。


|  --| you | say | goodbye | and | i | hello | . |
|:---:|:---:|:---:|:---:|:---:|:---:|:---:|:---:|
| you | 0 | 1 | 0 | 0 | 0 | 0 | 0 |
| say | 1 | 0 | 1 | 0 | 1 | 1 | 0 |
| goodbye | 0 | 1 | 0 | 1 | 0 | 0 | 0 |
| and | 0 | 0 | 1 | 0 | 1 | 0 | 0 |
| i | 0 | 1 | 0 | 1 | 0 | 0 | 0 |
| hello | 0 | 1 | 0 | 0 | 0 | 0 | 1 |
| . | 0 | 0| 0 | 0 | 0 | 1 | 0 |

上記の共起行列をプログラミングで自動で作成する関数を作成する。

In [23]:
import numpy as np

# vocab_size: 語彙数
def create_to_matrix(corpus, vocab_size, window_size=1):
    corpus_size = len(corpus)
    co_matrix = np.zeros((vocab_size, vocab_size), dtype=np.int32)
    
    for idx, word_id in enumerate(corpus):
        for i in range(1, window_size+1):
            left_idx = idx - i
            right_idx = idx + i
            
            if left_idx >= 0:
                left_word_id = corpus[left_idx]
                co_matrix[word_id, left_word_id] += 1
                
            if right_idx < corpus.size:
                right_word_id = corpus[right_idx]
                co_matrix[word_id, right_word_id] += 1
                
    return co_matrix
        
co_matrix = create_to_matrix(corpus, len(word_to_id))

print(co_matrix)

[[0 1 0 0 0 0 0]
 [1 0 1 0 1 1 0]
 [0 1 0 1 0 0 0]
 [0 0 1 0 1 0 0]
 [0 1 0 1 0 0 0]
 [0 1 0 0 0 0 1]
 [0 0 0 0 0 1 0]]


### 2.2.5 ベクトル間の類似度

単語のベクトル表現の類似度に関しては、コサイン類似度がよく用いられる。

```
similarity(x,y) = x・y / ||x|| ||y|| = x1y1 + ... + xnyx / √ x1^2 + ... xn^2 √ y1^2 + ... yn ^2
```
